In [1]:
import numpy as np
import pandas as pd
import networkx as nx

In [2]:
df = pd.read_csv('Final_dataset.csv')
y = df['author_id'].unique()
l = []
for i in y:
    df1 = df[df['author_id']==i]
    l.append(min(df1['year']))
    
d = {
    'author_id':y,
    'first_paper':l
}
df1 = pd.DataFrame(d)
df1 = pd.merge(df1,df)
df1['paper_diff'] = df1['year'] -  df1['first_paper']
first3 = df1[df1['paper_diff']<3]
first5 = df1[df1['paper_diff']<5]

In [3]:
first3.dtypes

author_id                 float64
first_paper               float64
accepted                  float64
citations                 float64
doc_id                    float64
rounds                    float64
year                      float64
degree_centrality         float64
page_rank                 float64
cluster_coeff             float64
closeness_centrality      float64
betweenness_centrality    float64
team_size                   int64
mean_citation             float64
Author_Reputation         float64
diff                       object
Sentiment                 float64
paper_diff                float64
dtype: object

In [3]:
l1 = first3['doc_id'].unique()

citer =[]
cited = []
with open('docidCitesDocid.txt') as infile:
    for line in infile:
        a = line.split()
        if int(a[0]) in l1:
            if int(a[1]) in l1:
                citer.append(a[0])
                cited.append(a[1])
        
d = {'citer' : citer,
     'cited' : cited}
df = pd.DataFrame(d)

In [4]:
doc =[]
auth = []
with open('DocIdAuthorId.txt') as infile:
    for line in infile:
        a = line.split()
        doc.append(a[0])
        auth.append(a[1])
        
d = {'citer' : doc,
     'citer_author' : auth}
df1 = pd.DataFrame(d)
df = pd.merge(df,df1)

doc =[]
auth = []
with open('DocIdAuthorId.txt') as infile:
    for line in infile:
        a = line.split()
        doc.append(a[0])
        auth.append(a[1])
        
d = {'cited' : doc,
     'cited_author' : auth}
df1 = pd.DataFrame(d)
df = pd.merge(df,df1)
df = df[['citer_author','cited_author']]

df = df.drop_duplicates()

In [5]:
G = nx.DiGraph()
for i,r in df.iterrows():
    G.add_edge(r['citer_author'],r['cited_author'])

In [6]:
G.remove_edges_from(G.selfloop_edges())
core = nx.algorithms.core_number(G)
y = first3['author_id'].unique()

cn = []
for i in y:
    try:
        cn.append(core[str(i)])
    except:
        cn.append(0)
        
d = {
    'author_id':y,
    'core':cn
}

core = pd.DataFrame(d)
core.to_csv('author_core_cocit.csv',index=False)

In [7]:
df.to_csv('Cocitation_network.csv',index=False)

In [8]:

pagerank = nx.pagerank(G)
id = pagerank.keys()
pr = pagerank.values()
d = {
    'Id' : id,
    'pagerank_cocit':pr
}
pagerank = pd.DataFrame(d)
pagerank.to_csv('pagerankcocitfirst3.csv',index = False)


In [ ]:


betweenness_centrality = nx.betweenness_centrality(G)
id = betweenness_centrality.keys()
bwn = betweenness_centrality.values()
d = {
    'Id' : id,
    'betweenness_centrality_cocit':bwn
}
betweenness_cen = pd.DataFrame(d)
betweenness_cen.to_csv('betweenness_centrality_cocitfirst5.csv',index = False)


In [23]:
G = nx.MultiDiGraph()
for i,r in df.iterrows():
    G.add_edge(r['citer_author'],r['cited_author'])

In [9]:
from networkx.algorithms.reciprocity import reciprocity

In [24]:
a = G.nodes()

In [25]:
reci = reciprocity(G,a)

In [26]:
id = reci.keys()
recip = reci.values()
d = {
    'Id' : id,
    'reciprocity':recip
}
recip = pd.DataFrame(d)
recip.to_csv('reciprocityfirst5.csv',index = False)

In [27]:
degree_centrality = nx.degree_centrality(G)
id = degree_centrality.keys()
degree = degree_centrality.values()
d = {
    'Id' : id,
    'degree_centrality':degree
}
degree_cen = pd.DataFrame(d)
degree_cen.to_csv('degree_centrality_cocitation_first5.csv',index = False)